In [316]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from pandas_profiling import ProfileReport
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

### Steps towards the Recommender System 
* Feature Engineering
* Classification Algorithm to predict "verified" column
* Recommender System

### Notes about the features
* unixReviewTime --> will be useful for a classification algorithm in order to predict the "verified" feature.
* reviewerName --> also seems useless both for the classification and the recommender system algorithms, as generally every family only has one game instead of many for each member.
* Image --> won't be used for any of the models, as it contains way too many null values (99% approximately).
* asin --> contains 266 unique values. It might contain useful information for the classification algorithm as, in relation to the size of the dataset, this number is still pretty small. Obviously won't be used by the Recommender System.
* reviewerID --> It has also been observed that approximately 20% of the reviews were made by users who had already reviewed another product before. However 80% weren't. Transforming this column into a dummy "Reviewed Twice or Not kind of user" feature might be useful!!!
* reviewTime --> might be useful for the classification algorithm, however the information on it will be divided in order to include month, year, and day on separate columns. Reason for this is the seasonality factor. Perhaps on certain periods of the month or year there is a stronger tendency for people to verify less or more a specific review. 
* Overall --> It might be useful, perhaps how good a specific product was evaluated has a correlation with it being or not being verified, will be used for both classification AND recommender system algorithm.

In [227]:
VideoGamesDS = pd.read_json('Video_Games_sample.json',lines=True)
VideoGamesDS

,overall,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,verified,vote,image
0,1,"01 2, 2018",A16FEXIKAPT24U,B00000JRSB,{'Format:': ' Video Game'},Nicholas Sabin,Game disc was cracked thats pretty lame,Game disc 1 was cracked,1514851200,NaN,NaN,NaN
1,5,"01 2, 2018",A3ISBEPYLY8IMO,B00000JRSB,{'Format:': ' Video Game'},Jimmy,Received this in perfect condition. Great rese...,Received this in perfect condition. Great rese...,1514851200,1.0,NaN,NaN
2,5,"12 21, 2017",A1RIUB1PZWLOVZ,B00000JRSB,{'Format:': ' Video Game'},Amazon Customer,I had a hard time finding this game locally or...,I am very pleased with this purchase,1513814400,1.0,NaN,NaN
3,5,"12 20, 2017",A2VY0K54SOCG0F,B00000JRSB,{'Format:': ' Video Game'},Oscentatious,This game is just as amazing as the first time...,This is the best deal for this game,1513728000,0.0,NaN,NaN
4,5,"12 19, 2017",A1ABJQ2REV14OQ,B00000JRSB,{'Format:': ' Video Game'},Thiago,"I bought as a gift for my friend, she is so ex...",Loved by lots of people,1513641600,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
453557,5,"09 30, 2018",A3QBMO3NTKUU21,B01H6GUCCQ,{'Color:': ' Blue'},Stephanie McCoy,"Works perfectly, cord is nice and long, the LE...",High Quality Headset,1538265600,1.0,NaN,NaN
453558,1,"09 30, 2018",A3ELGOLQ8393WC,B01H6GUCCQ,{'Color:': ' Red'},James,Disappointed. Got this for my sons birthday an...,Broke within a week,1538265600,1.0,NaN,NaN
453559,2,"09 29, 2018",A2MMNV4U0G39S6,B01H6GUCCQ,{'Color:': ' Red'},Heather,My boys loved these but both broke after only ...,Not durable,1538179200,1.0,NaN,NaN
453560,1,"09 29, 2018",A1GFKSMVXX3G0X,B01H6GUCCQ,{'Color:': ' Red'},TJ,Bought this for my son to use on his Xbox. Mic...,Mic rarely works,1538179200,NaN,NaN,NaN


In [228]:
VideoGamesDS['reviewerID'].nunique()/VideoGamesDS.shape[0]

0.7965570307918212

In [229]:
reviewIDsVCounts = pd.DataFrame(VideoGamesDS['reviewerID'].value_counts())

In [230]:
reviewIDsVCounts.loc['A29BQ6B90Y1R5F',:][0]

34

In [233]:
l =[]
for i in VideoGamesDS['reviewerID']:
    l.append(reviewIDsVCounts.loc[i,:][0])
    if len(l)%100000 ==0:
        print(f'Linha sendo substituida: {len(l)}')
VideoGamesDS['reviewerID'] = l
VideoGamesDS.rename(columns={'reviewerID':'NTimesReviewed'},inplace=True)

Linha sendo substituida: 100000
Linha sendo substituida: 200000
Linha sendo substituida: 300000
Linha sendo substituida: 400000


In [234]:
VideoGamesDS['Month'] = pd.to_numeric(VideoGamesDS['reviewTime'].str[:2])
VideoGamesDS['Year'] = pd.to_numeric(VideoGamesDS['reviewTime'].str[-4:])

In [235]:
l = []
for i in VideoGamesDS['reviewTime']:
    l.append(i[i.find(' '):i.find(',')])
    if len(l)%100000 == 0:
        print(f'Linha sendo substituida: {len(l)}')
VideoGamesDS['Day'] = l

Linha sendo substituida: 100000
Linha sendo substituida: 200000
Linha sendo substituida: 300000
Linha sendo substituida: 400000


In [237]:
VideoGamesDS['vote'].isna().sum()/VideoGamesDS.shape[0]

0.8740569095294579

In [238]:
VideoGamesDS['overall'].nunique()

5

In [239]:
len(VideoGamesDS[VideoGamesDS.verified == 1])/len(VideoGamesDS[VideoGamesDS.verified == 0])

4.470382383065893

In [244]:
VideoGamesDSClass = VideoGamesDS.sample(10000)
VideoGamesDSClass = pd.concat([VideoGamesDSClass[VideoGamesDSClass.verified == 1].sample(4000),
                              VideoGamesDSClass[VideoGamesDSClass.verified == 0]])
VideoGamesDSClass = pd.concat([VideoGamesDSClass.drop(columns=['asin','style']),pd.get_dummies(VideoGamesDSClass[['asin']])],1)
X = VideoGamesDSClass[VideoGamesDSClass['verified'].notna()].drop(columns=['vote','reviewerName','reviewText','summary','image','reviewTime','verified'])
y = VideoGamesDSClass[VideoGamesDSClass['verified'].notna()]['verified']
kf = KFold(n_splits=5)
kf.get_n_splits(X)
DFFinal = pd.DataFrame(index=[['Precision','Accuracy','F1','Recall']])
for train_index, test_index in kf.split(X):
    print('\n\n----- Novo Cross Validation -----\n\n')
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    scaler = MinMaxScaler()
    scaler = scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train),columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test),columns=X_test.columns)
    
    DF = pd.DataFrame(index=[['Precision','Accuracy','F1','Recall']])

    Modelos = {'Random Forest':RandomForestClassifier(),
              'Gradient Boosting':GradientBoostingClassifier(),
              'MLP Classifier':MLPClassifier(),
              'Naive Bayes Classifier':GaussianNB()}

    
    for keys,values in Modelos.items():
        print(f'Modelo sendo testado: {keys}')
        Predictions = values.fit(X_train,y_train)
        Predictions = Predictions.predict(X_test)
        List = [precision_score(Predictions,y_test), accuracy_score(Predictions,y_test),
                f1_score(Predictions,y_test),recall_score(Predictions,y_test)]
        DF = pd.concat([DF,pd.DataFrame(List,index=[['Precision','Accuracy','F1','Recall']],columns=[keys])],1)
    DFFinal = pd.concat([DFFinal,DF],1)
    print(DF)
print('\n\n----- Comparação Final Entre Modelos -----\n\n')
DFFinal.groupby(lambda x:x, axis=1).sum()/4



----- Novo Cross Validation -----


Modelo sendo testado: Random Forest
Modelo sendo testado: Gradient Boosting
Modelo sendo testado: MLP Classifier
Modelo sendo testado: Naive Bayes Classifier
           Random Forest  Gradient Boosting  MLP Classifier  \
Precision       0.908273           0.930755        0.842626   
Accuracy        0.908273           0.930755        0.842626   
F1              0.951932           0.964136        0.914592   
Recall          1.000000           1.000000        1.000000   

           Naive Bayes Classifier  
Precision                0.346223  
Accuracy                 0.346223  
F1                       0.514362  
Recall                   1.000000  


----- Novo Cross Validation -----


Modelo sendo testado: Random Forest
Modelo sendo testado: Gradient Boosting
Modelo sendo testado: MLP Classifier
Modelo sendo testado: Naive Bayes Classifier
           Random Forest  Gradient Boosting  MLP Classifier  \
Precision       0.891089           0.927993      

,Gradient Boosting,MLP Classifier,Naive Bayes Classifier,Random Forest
Precision,0.931750,0.838673,0.341658,0.910375
Accuracy,0.952086,0.883699,0.607258,0.935439
F1,0.926130,0.878720,0.502816,0.916711
Recall,0.931200,0.936803,0.956364,0.933858


It was important to reduce the sample size as it was taking way too long to perform the Grid Search.

In [249]:
VideoGamesDSClass = VideoGamesDS.sample(1500)
VideoGamesDSClass = pd.concat([VideoGamesDSClass[VideoGamesDSClass.verified == 1].sample(600),
                              VideoGamesDSClass[VideoGamesDSClass.verified == 0]])
VideoGamesDSClass = pd.concat([VideoGamesDSClass.drop(columns=['asin','style']),pd.get_dummies(VideoGamesDSClass[['asin']])],1)
X = VideoGamesDSClass[VideoGamesDSClass['verified'].notna()].drop(columns=['vote','reviewerName','reviewText','summary','image','reviewTime','verified'])
y = VideoGamesDSClass[VideoGamesDSClass['verified'].notna()]['verified']
Grid = {
    'n_estimators':[100,200,300],
    'criterion':['friedman_mse','squared_error','mse'],
    'max_depth':[3,5,8]
}

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

scaler = MinMaxScaler().fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train),columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test),columns = X_test.columns)

Nana = GridSearchCV(GradientBoostingClassifier(),param_grid=Grid,cv=3,n_jobs = -1, verbose = 2).fit(X_train,y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


In [250]:
Nana.best_estimator_

GradientBoostingClassifier(criterion='squared_error')

In [253]:
VideoGamesDSClass = pd.concat([VideoGamesDS.drop(columns=['asin','style']),pd.get_dummies(VideoGamesDS[['asin']])],1)

X_test = VideoGamesDSClass[VideoGamesDSClass['verified'].isna()].drop(columns=['vote','reviewerName','reviewText','summary','image','reviewTime','verified'])
X_train = VideoGamesDSClass[VideoGamesDSClass['verified'].notna()].drop(columns=['vote','reviewerName','reviewText','summary','image','reviewTime','verified'])
y_train = VideoGamesDSClass[VideoGamesDSClass['verified'].notna()]['verified']

scaler = MinMaxScaler().fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train),columns= X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test),columns=X_test.columns)

Predictor = Nana.best_estimator_.fit(X_train,y_train)
Results = Predictor.predict(X_test)

print(Results)

[1. 1. 1. ... 1. 1. 1.]


In [280]:
a = 0
l = []
for i in range(len(VideoGamesDS['verified'])):
    if i%10000 == 0:
        print(f'Line being replaced: {i}')
    if VideoGamesDS['verified'].isna()[i] == True:
        l.append(Results[a])
        a += 1
    else:
        l.append(VideoGamesDS['verified'][i])
print(len(l))
VideoGamesDS['verified'] = l

Line being replaced: 0
Line being replaced: 10000
Line being replaced: 20000
Line being replaced: 30000
Line being replaced: 40000
Line being replaced: 50000
Line being replaced: 60000
Line being replaced: 70000
Line being replaced: 80000
Line being replaced: 90000
Line being replaced: 100000
Line being replaced: 110000
Line being replaced: 120000
Line being replaced: 130000
Line being replaced: 140000
Line being replaced: 150000
Line being replaced: 160000
Line being replaced: 170000
Line being replaced: 180000
Line being replaced: 190000
Line being replaced: 200000
Line being replaced: 210000
Line being replaced: 220000
Line being replaced: 230000
Line being replaced: 240000
Line being replaced: 250000
Line being replaced: 260000
Line being replaced: 270000
Line being replaced: 280000
Line being replaced: 290000
Line being replaced: 300000
Line being replaced: 310000
Line being replaced: 320000
Line being replaced: 330000
Line being replaced: 340000
Line being replaced: 350000
Line b

In [289]:
VGDS = VideoGamesDS[(VideoGamesDS['verified'] == 0)|(VideoGamesDS['verified']==1)]

In [291]:
VGamesDS = pd.read_json('Video_Games_sample.json',lines=True)
VGDS['ReviewerID'] = VGamesDS['reviewerID']

In [298]:
VGDS

,overall,reviewTime,NTimesReviewed,asin,style,reviewerName,reviewText,summary,unixReviewTime,verified,vote,image,Month,Year,Day,ReviewerID
0,1,"01 2, 2018",1,B00000JRSB,{'Format:': ' Video Game'},Nicholas Sabin,Game disc was cracked thats pretty lame,Game disc 1 was cracked,1514851200,1.0,NaN,NaN,1,2018,2,A16FEXIKAPT24U
1,5,"01 2, 2018",3,B00000JRSB,{'Format:': ' Video Game'},Jimmy,Received this in perfect condition. Great rese...,Received this in perfect condition. Great rese...,1514851200,1.0,NaN,NaN,1,2018,2,A3ISBEPYLY8IMO
2,5,"12 21, 2017",1,B00000JRSB,{'Format:': ' Video Game'},Amazon Customer,I had a hard time finding this game locally or...,I am very pleased with this purchase,1513814400,1.0,NaN,NaN,12,2017,21,A1RIUB1PZWLOVZ
4,5,"12 19, 2017",1,B00000JRSB,{'Format:': ' Video Game'},Thiago,"I bought as a gift for my friend, she is so ex...",Loved by lots of people,1513641600,1.0,NaN,NaN,12,2017,19,A1ABJQ2REV14OQ
5,5,"12 9, 2017",3,B00000JRSB,{'Format:': ' Video Game'},Jonathan carter,"Looks good, some scratches on discs but plays.",Five Stars,1512777600,1.0,NaN,NaN,12,2017,9,A368DF2G0T502X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453557,5,"09 30, 2018",1,B01H6GUCCQ,{'Color:': ' Blue'},Stephanie McCoy,"Works perfectly, cord is nice and long, the LE...",High Quality Headset,1538265600,1.0,NaN,NaN,9,2018,30,A3QBMO3NTKUU21
453558,1,"09 30, 2018",1,B01H6GUCCQ,{'Color:': ' Red'},James,Disappointed. Got this for my sons birthday an...,Broke within a week,1538265600,1.0,NaN,NaN,9,2018,30,A3ELGOLQ8393WC
453559,2,"09 29, 2018",1,B01H6GUCCQ,{'Color:': ' Red'},Heather,My boys loved these but both broke after only ...,Not durable,1538179200,1.0,NaN,NaN,9,2018,29,A2MMNV4U0G39S6
453560,1,"09 29, 2018",1,B01H6GUCCQ,{'Color:': ' Red'},TJ,Bought this for my son to use on his Xbox. Mic...,Mic rarely works,1538179200,1.0,NaN,NaN,9,2018,29,A1GFKSMVXX3G0X


In [293]:
VGDS = VGDS[VGDS['verified']==1]
VGDS[['overall','ReviewerID','asin']]

,overall,ReviewerID,asin
0,1,A16FEXIKAPT24U,B00000JRSB
1,5,A3ISBEPYLY8IMO,B00000JRSB
2,5,A1RIUB1PZWLOVZ,B00000JRSB
4,5,A1ABJQ2REV14OQ,B00000JRSB
5,5,A368DF2G0T502X,B00000JRSB
...,...,...,...
453557,5,A3QBMO3NTKUU21,B01H6GUCCQ
453558,1,A3ELGOLQ8393WC,B01H6GUCCQ
453559,2,A2MMNV4U0G39S6,B01H6GUCCQ
453560,1,A1GFKSMVXX3G0X,B01H6GUCCQ


In [306]:
VGDS[['overall','asin']].groupby('asin').mean()\
.sort_values(by='overall',ascending=False)\
.rename(columns={'overall':'Average Rating'})

,Average Rating
asin,
B00UG63VDG,4.793137
B00AKIPBNS,4.793103
B0017KIBAI,4.788868
B00DC7G0GG,4.781609
B00K848IH0,4.768563
...,...
B000XGJH1O,2.920290
B00ZJRHSZO,2.747208
B00ZQB28XK,2.720599


In [389]:
Games_Sold = VGDS[['overall','asin']].groupby('asin').count()\
.sort_values(by='overall',ascending=False)\
.rename(columns={'overall':'Games Sold'})

In [320]:
Games_Matrix = VGDS[['overall','asin','ReviewerID']] \
.pivot_table(index='asin',columns='ReviewerID',values='overall',aggfunc='sum') \
.applymap(lambda x: 1 if x>0 else 0)

In [321]:
Sim_Games = pd.DataFrame(cosine_similarity(Games_Matrix),index=Games_Matrix.index, columns = Games_Matrix.index)

In [338]:
ItemNeighbours = pd.DataFrame(index=Sim_Games.columns,columns=range(1,10))
for i in range(0,len(Sim_Games.columns)): 
    ItemNeighbours.iloc[i,:9] = Sim_Games.iloc[0:,i].sort_values(ascending=False)[:9].index

In [368]:
NewMatrix = Games_Matrix.transpose()
NewMatrix

asin,B00000JRSB,B00005NZ1G,B00005O0I2,B00005Q8M0,B00005TNI6,B000066TS5,B0000696CZ,B000087H7T,B00008J7NZ,B00009OY9U,...,B01DPS4QQ2,B01EJ9DMQQ,B01EZAA2ZI,B01F84ZHMI,B01FZ3BR5S,B01GW3ODBU,B01GW3P9PE,B01H1GJ7IQ,B01H482N6E,B01H6GUCCQ
ReviewerID,,,,,,,,,,,,,,,,,,,,,
A0003214FKMKJE0PCW3D,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0005622E33Y7RCVKKVN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00065507CNSR8UHQFCK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00101847G3FJTWYGNQA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0011756FPL8K71Q5TAQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZZWWSMIDFU6S,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AZZX40NRMUDDQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AZZX6JJHIO4UZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [374]:
CustItemSimilarity = pd.DataFrame(index=NewMatrix.index,columns=NewMatrix.columns) 
def getScore(history, similarities):
    return sum(history*similarities)/sum(similarities)

In [399]:
for i in range(0,len(CustItemSimilarity.index[:1000])): 
    if i%100 ==0:
        print(f'Index being tested: {i}')
    for j in range(0,len(CustItemSimilarity.columns)): 
        user = CustItemSimilarity.index[i] 
        product = CustItemSimilarity.columns[j]
        if NewMatrix.iloc[i][j] > 0: 
            CustItemSimilarity.iloc[i][j] = 0 
        else: 
            ItemTop = ItemNeighbours.loc[product][1:9]
            ItemTopSimilarity = Sim_Games.loc[product].sort_values(ascending=False)[1:9] 
            CustomerPurchasings = NewMatrix.loc[user,ItemTop]
            CustItemSimilarity.iloc[i][j] = getScore(CustomerPurchasings,ItemTopSimilarity) 

Index being tested: 0
Index being tested: 100
Index being tested: 200
Index being tested: 300
Index being tested: 400
Index being tested: 500
Index being tested: 600
Index being tested: 700
Index being tested: 800
Index being tested: 900


In [400]:
CustItemRecommend = pd.DataFrame(index=CustItemSimilarity.index[:1000], columns=['Customer','1','2','3','4','5','6']) 
for i in range(0,len(CustItemSimilarity.index[:1000])): 
    CustItemRecommend.iloc[i,1:] = CustItemSimilarity.iloc[i,:].sort_values(ascending=False).iloc[1:7,].index.transpose() 

,Customer,1,2,3,4,5,6
ReviewerID,,,,,,,
A0003214FKMKJE0PCW3D,NaN,B00KKAQYXM,B00FNKMVUO,B00HTK1NCS,B00E290JRE,B00GU8W5AE,B00IRHE892
A0005622E33Y7RCVKKVN,NaN,B00JJNQG98,B00CD1FC6G,B00CQ35C1Q,B00AOIRCI6,B0068INSUM,B013HSWF40
A00065507CNSR8UHQFCK,NaN,B0013OL0BK,B004IK24MU,B00434FED2,B002BRZ9G0,B000X2RKOO,B000YDIA78
A00101847G3FJTWYGNQA,NaN,B000NUBY0C,B000087H7T,B000TLU67W,B00000JRSB,B00IVJ1M7M,B00IRHE892
A0011756FPL8K71Q5TAQ,NaN,B000NUBY0C,B000087H7T,B002BRZ9G0,B002BSA3EM,B00000JRSB,B00IFRH6JS


In [405]:
CustItemRecommend

,Customer,1,2,3,4,5,6
ReviewerID,,,,,,,
A0003214FKMKJE0PCW3D,NaN,B00KKAQYXM,B00FNKMVUO,B00HTK1NCS,B00E290JRE,B00GU8W5AE,B00IRHE892
A0005622E33Y7RCVKKVN,NaN,B00JJNQG98,B00CD1FC6G,B00CQ35C1Q,B00AOIRCI6,B0068INSUM,B013HSWF40
A00065507CNSR8UHQFCK,NaN,B0013OL0BK,B004IK24MU,B00434FED2,B002BRZ9G0,B000X2RKOO,B000YDIA78
A00101847G3FJTWYGNQA,NaN,B000NUBY0C,B000087H7T,B000TLU67W,B00000JRSB,B00IVJ1M7M,B00IRHE892
A0011756FPL8K71Q5TAQ,NaN,B000NUBY0C,B000087H7T,B002BRZ9G0,B002BSA3EM,B00000JRSB,B00IFRH6JS
...,...,...,...,...,...,...,...
A10CBQVSCTVVPE,NaN,B001FY7LBQ,B005WWZUQ0,B001CXYMFS,B001O5CCQK,B003VAHYQY,B001NT9TK4
A10CC9IFSDECEG,NaN,B003VAHYQY,B00OAYHIRA,B019H5II8Y,B0076HD2W8,B00E290JRE,B009IR1SR0
A10CCR309RHCMP,NaN,B0096PLB9O,B004774IPU,B00009OY9U,B00GANWVJE,B00GMFKYK8,B00GODZYNA
